In [ ]:
# reference: https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb
# tutorial: https://www.youtube.com/watch?v=pxhkDaKzBaY

In [2]:
# # Use for colab only
# %%capture
# %pip install unsloth==2025.3.6 unsloth_zoo==2025.3.4

In [3]:
import os
os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
from unsloth import FastLanguageModel
import torch
import json

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


[2025-04-16 10:42:43,706] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using

In [ ]:
with open('key.json', encoding='utf-8') as f:
    token = json.load(f)['hf_key']

max_seq_length = 2048
dtype = None
load_in_4bit = True
model_name = 'utter-project/EuroLLM-1.7B'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    token = token,
    device_map="auto",
)

c:\Users\Artis\AppData\Local\Programs\Python\Python310\lib\site-packages\unsloth_zoo\gradient_checkpointing.py:330: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:28.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"cuda:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.3.19: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    NVIDIA GeForce RTX 2060. Num GPUs = 1. Max memory: 6.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.6.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
utter-project/EuroLLM-1.7B does not have a padding token! Will use pad_token = <unk>.


In [6]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2025.3.19 patched 24 layers with 24 QKV layers, 24 O layers and 24 MLP layers.


In [ ]:
from datasets import Dataset
from unsloth import to_sharegpt, standardize_sharegpt, apply_chat_template
def PrepareDataset(pathToFile):
    with open(pathToFile, encoding='utf-8') as f:
        data = json.load(f)

    dataset = Dataset.from_dict({
        "instruction": [entry['instruction'] for entry in data],
        "output": [entry['output'] for entry in data],
    })

    dataset = to_sharegpt(
        dataset,
        merged_prompt = "{instruction}",
        output_column_name = "output",
        conversation_extension = 3, # Select more to handle longer conversations
    )

    dataset = standardize_sharegpt(dataset) 

    chat_template = """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.

    ### Instruction:
    {INPUT}

    ### Response:
    {OUTPUT}"""

    dataset = apply_chat_template(
        dataset,
        tokenizer = tokenizer,
        chat_template = chat_template,
        # default_system_message = "You are a helpful assistant", << [OPTIONAL]
    )

    return dataset

In [ ]:
path_to_training_set = 'datasets/instruction_dataset_14873.json'
train_dataset = PrepareDataset(path_to_training_set)
eval_dataset = PrepareDataset('datasets/validation_dataset.json')

Merging columns:   0%|          | 0/14873 [00:00<?, ? examples/s]

Converting to ShareGPT:   0%|          | 0/14873 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/14873 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/14873 [00:00<?, ? examples/s]

Flattening the indices:   0%|          | 0/14873 [00:00<?, ? examples/s]

...

Flattening the indices:   0%|          | 0/260 [00:00<?, ? examples/s]

Extending conversations:   0%|          | 0/260 [00:00<?, ? examples/s]

Unsloth: Standardizing formats (num_proc=20):   0%|          | 0/260 [00:00<?, ? examples/s]

Unsloth: We automatically added an EOS token to stop endless generations.


Map:   0%|          | 0/260 [00:00<?, ? examples/s]

In [9]:
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

learning_rate = 3e-4
epochs = 5
steps = 3720

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    eval_dataset = eval_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 1,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        max_steps = steps,
        warmup_steps = 300,
        # num_train_epochs = epochs,
        learning_rate = learning_rate,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = f"checkpoints/{model_name}",
        # save_strategy = "epoch",
        save_strategy = "steps",
        save_steps = 100,
        eval_strategy = 'steps', # validēšanu veic pa soļiem
        eval_steps = 100 # ik pa cik soļiem veic validēšanu? Attēlo trenēšanas un validēšanas rezultātus.
        # eval_strategy = 'epoch', # validēšanu veic pa soļiem
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Unsloth: Tokenizing ["text"]:   0%|          | 0/14873 [00:00<?, ? examples/s]

Unsloth: Tokenizing ["text"]:   0%|          | 0/260 [00:00<?, ? examples/s]

In [10]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 2060. Max memory = 6.0 GB.
1.605 GB of memory reserved.


In [11]:
try:
    trainer_stats = trainer.train(resume_from_checkpoint = True)
except:
    trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 14,873 | Num Epochs = 3 | Total steps = 3,720
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 7,176,192/7,000,000,000 (0.10% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss


In [12]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

12.2093 seconds used for training.
0.2 minutes used for training.
Peak reserved memory = 3.051 GB.
Peak reserved memory for training = 1.446 GB.
Peak reserved memory % of max memory = 50.85 %.
Peak reserved memory for training % of max memory = 24.1 %.


In [ ]:
model.save_pretrained_gguf(f"EuroLLM/EuroLLM-LatLeg-1.7B-14.8K", tokenizer, quantization_method = ['f16'])

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### Your chat template has a BOS token. We shall remove it temporarily.


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 9.85 out of 31.92 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 24/24 [00:00<00:00, 44.28it/s]


Unsloth: Saving tokenizer... Done.
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m', 'f16'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: [1] Converting model at EuroLLM/EuroLLM-LatLeg-1.7B-14.8K into f16 GGUF format.
The output location will be c:\Users\Artis\Documents\GitHub\LatLegModel\ModelFineTuningPipeline\EuroLLM\EuroLLM-LatLeg-1.7B-14.8K\unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: EuroLLM-LatLeg-1.7B-14.8K
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model part 'model.safetensors'
INFO:hf-to-gguf:output.weight,               torch.float16 

Unsloth: ##### The current model auto adds a BOS token.
Unsloth: ##### We removed it in GGUF's chat template for you.


Unsloth: Conversion completed! Output location: c:\Users\Artis\Documents\GitHub\LatLegModel\ModelFineTuningPipeline\EuroLLM\EuroLLM-LatLeg-1.7B-14.8K\unsloth.Q4_K_M.gguf
Unsloth: Saved Ollama Modelfile to EuroLLM/EuroLLM-LatLeg-1.7B-14.8K\Modelfile


In [ ]:
# print(tokenizer._ollama_modelfile)

FROM {__FILE_LOCATION__}

TEMPLATE """Below are some instructions that describe some tasks. Write responses that appropriately complete each request.{{ if .Prompt }}

    ### Instruction:
    {{ .Prompt }}{{ end }}

    ### Response:
    {{ .Response }}</s>"""

PARAMETER stop "<|im_start|>"
PARAMETER stop "<unk>"
PARAMETER stop "<|im_end|>"
PARAMETER stop "</s>"
PARAMETER stop "<extra_id_"
PARAMETER temperature 1.5
PARAMETER min_p 0.1


In [ ]:
# !ollama create EuroLLM-1.7B-LatLeg-14.8K-Q8 -f ./EuroLLM/EuroLLM-LatLeg-1.7B-14.8K/Modelfile

gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering model components ⠼ gathering model components ⠴ gathering model components ⠦ gathering model components ⠧ gathering model components ⠇ gathering model components ⠏ gathering model components ⠋ gathering model components ⠙ gathering model components ⠹ gathering model components ⠸ gathering mode